In [1]:
%load_ext jupyter_black

<div style="display: flex; align-items: center;">
  <div style="flex: 1;">
    <img src="images/notebook.gif" alt="segment" width="500">
  </div>
  <div style="flex: 1;">
    <h2> Practical Prompt Engineering </h2>
    <br>
    <div>
       In this tutorial, you’ll learn how to:
       <ul>
        <li>Apply prompt engineering techniques to practical, real-world examples
        <li>Tap into the power of roles in messages to go beyond using singular role prompts
        <li>Use numbered steps, delimiters, few-shot prompting and other techniques to improve your results
        <li>Understand and use chain-of-thought prompting to add more context
      </ul>
    <div>

  </div>
</div>

First let's import all the libraries we're going to use...

In [2]:
import os
import openai

and retrieve our OpenAI API key...

In [64]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

Chat completion API [documentation](https://platform.openai.com/docs/guides/gpt):

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020.",
        },
    ],
)

Chat completions [response](https://platform.openai.com/docs/guides/gpt/chat-completions-response-format) format:

In [97]:
response = {
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "message": {
                "content": "The Los Angeles Dodgers won the World Series in 2020.",
                "role": "assistant",
            },
        }
    ],
    "created": 1677664795,
    "id": "chatcmpl-7QyqpwdfhqwajicIEznoc6Q47XAyW",
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "usage": {"completion_tokens": 17, "prompt_tokens": 57, "total_tokens": 74},
}

The assistant’s reply can be extracted with:

In [ ]:
response["choices"][0]["message"]["content"]

The next function allows us to send prompts to the model and get a response back. <br>

We have specified the following model parameters:
* Model: **GPT-3.5-turbo**
* Temperature: **0**

The function takes as inputs <u>prompt</u> and a <u>list of previous messages</u>, and returns a new list of messsages that include the prompt and the response from the model:

In [ ]:
def get_completion_from_messages(
    prompt,
    messages,
    model="gpt-3.5-turbo",
    temperature=0
):

    if not isinstance(messages, list):
        messages = [messages]

    messages.append(prompt)
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    print(response.choices[0].message["content"])

    messages.append(response.choices[0].message)
    return messages

Let's test it!

In [ ]:
messages = [
    {"role": "user", "content": "My name is Alex"},
]

prompt = {"role": "user", "content": "What is my name?"}

response = get_completion_from_messages(prompt, messages)

In [41]:
def fprint(output):
    if isinstance(output, list):
        for item in output:
            if "role" in item and "content" in item:
                role = item["role"]
                content = item["content"].strip()

                if role == "user":
                    prefix = "User"
                elif role == "assistant":
                    prefix = "Assistant"
                    content = content.replace(
                        "\n\n", "\n"
                    )  # Replace double newlines with single newlines
                else:
                    prefix = role.capitalize()

                formatted_item = f"{prefix}: \n {content}\n"
                print(formatted_item)
                print("---" * 30)
            else:
                print("Invalid output format")
    else:
        print("Invalid output format")

# Ways to Engineer your Prompts

We will now look at a few different examples of how you can improve your promtps to become the utlimate prompt engineer!

Let's explore the following techniques:

<ul>
<li> <b>Zero-shot prompting:</b> Asking the language model a normal question without any additional context
<li> <b>Role prompting:</b> Asking the language model a normal question without any additional context
<li> <b>Detail & Specificity:</b> Breaking down a complex prompt into a series of small, specific steps
<li> <b>Few-shot prompting:</b> Conditioning the model on a few examples to boost its performance
<li> <b>Using delimiters:</b> Adding special tokens or phrases to provide structure and instructions to the model
</ul>

## 1. Zero-shot prompting 🤖

First, we have a list of reviews written below. Our task is to summarise them using ChatGPT. <br> Let's see what we can do.

In [37]:
reviews = [
    (
        "Date: December 15, 2021; Username: John123; Review: I am absolutely delighted \
    with this savings product! The interest rates are fantastic, and it has helped \
    me grow my savings significantly. Highly recommend!"
    ),
    (
        "Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite \
    disappointed with this savings product. The promised returns were not as \
    impressive as advertised, and the fees associated with it added up quickly. \
    Not worth it."
    ),
    (
        "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather \
    indifferent about this savings product. It's just like any other basic savings \
    account out there. Nothing special, but it does the job."
    ),
]

In [76]:
content = f"""Summarise these 3 reviews:{reviews}"""

print(content)

Summarise these 3 reviews:['Date: December 15, 2021; Username: John123; Review: I am absolutely delighted     with this savings product! The interest rates are fantastic, and it has helped     me grow my savings significantly. Highly recommend!', 'Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite     disappointed with this savings product. The promised returns were not as     impressive as advertised, and the fees associated with it added up quickly.     Not worth it.', "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather     indifferent about this savings product. It's just like any other basic savings     account out there. Nothing special, but it does the job."]


In [48]:
prompt = {
    "role": "user",
    "content": content,
}

messages = get_completion_from_messages(prompt, [])

Review 1: John123 is extremely satisfied with the savings product, praising the fantastic interest rates and significant growth of their savings. Highly recommends it.

Review 2: Sarah77 is disappointed with the savings product, as the promised returns were not as impressive as advertised and the associated fees accumulated quickly. Considers it not worth it.

Review 3: AlexSmith expresses indifference towards the savings product, considering it similar to any other basic savings account. It is deemed as nothing special but gets the job done.


Great! It provided a summary of each review.

What we just did is call **zero-shot prompting**, which is just a fancy way of saying that you’re asking a normal question or simply describing a task.

But it's safe to say this is by no means useful enough to be a product yet. 

Let's now dive into the other prompt engineering techniques to improve our results. 

### System Role

The system message is a behind-the-scenes prompt sent to the model before the user begins querying. <br>
Typically we can use it to determine what **role** the AI should play and how it should behave generally. <br> 
For example, if you are creating a Pizza Ordering Chatbot, then you can specify that in the system message.

<div class="alert alert-block alert-info">

<b>Tip: </b>The 3 types of roles you can use are: <br>

<ul>
<li> <u>System role:</u> Allows you to specify the way the model answers questions. <br>
                 Classic example: “You are a helpful assistant.”

<li> <u>User role:</u> Equivalent to the queries made by the user

<li> <u>Assistant role:</u> The model’s responses
</ul>

</div>

## 2. Role Prompting 🤖

In the next example we are setting the **role** of the model via the system message. This is known as "Role Prompting", and is a general practice to help set the tone and context for the model's responses.

In [77]:
system_message = {
    "role": "system",
    "content": (
        """
        You are a Review Summariser. \
        Your job is to process reviews from customers, \
        and summarise them for analysis for the customer review team.
        """
    ),
}

messages = [system_message]

In [79]:
prompt = {
    "role": "user",
    "content": content,
}

messages = get_completion_from_messages(prompt, messages)

Review 1: John123 is highly satisfied with the savings product, praising the fantastic interest rates and significant growth of their savings. Highly recommends it.

Review 2: Sarah77 is disappointed with the savings product. The promised returns were not as impressive as advertised, and the fees associated with it added up quickly. Not worth it.

Review 3: AlexSmith is indifferent about the savings product. It is described as a basic savings account with nothing special, but it gets the job done.


<div class="alert alert-block alert-success">

By using a role prompt via the system message, the summary is more structured and much clearer to read for the user.
    
</div>

## 3. Detail and Specificity

It is important that when you prompt, you are specific and as clear as possible.

Note that clear ≠ short: the more detail you add, the better the model will understand how you want it to behave.

In the next example, we'll try to add a category for each review: positive, neutral or negative:

In [85]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser. \
    Your job is to process reviews from customers, \
    and summarise them for analysis for the customer review team.
    Categorise each review as positive, neutral, or bad.
    """,
}

prompt = {"role": "user", "content": content}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

Review 1: Positive
Review 2: Bad
Review 3: Neutral


As you can see, the model has only output the categories, and we're now missing all the other information. Let's be a bit more specific:

In [99]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser.
    Your job is to process reviews from customers
    and summarise them for analysis for the customer review team.

    For each review output the name and date. 
    Also label each review as either 'Positive','Neutral', or 'Negative'.
    Then provide a summary of key points in a single sentence.
    """,
}

prompt = {"role": "user", "content": content}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

Review 1:
- Name: John123
- Date: December 15, 2021
- Sentiment: Positive
- Summary: John123 is delighted with the savings product, praising the fantastic interest rates and significant growth of savings.

Review 2:
- Name: Sarah77
- Date: November 28, 2021
- Sentiment: Negative
- Summary: Sarah77 is disappointed with the savings product, mentioning that the promised returns were not as impressive as advertised and the fees added up quickly, making it not worth it.

Review 3:
- Name: AlexSmith
- Date: January 5, 2022
- Sentiment: Neutral
- Summary: AlexSmith is indifferent about the savings product, stating that it is like any other basic savings account and does the job.


Great! We now have all the information displayed appropriately for each review. It's even structured the data for us in a list! Let's see if we can use a structure then to formalise our output a bit better.

You can further increase your specificity by describing your request in Numbered Steps as well as specifying the format that you want the model to use for your output


In [102]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser.
    Your job is to process reviews from customers
    and summarise them for analysis for the customer review team:

    1. For each review output the name and date. 
    2. Also label each review as either 'Positive','Neutral', or 'Negative'.
    3. Provide a summary of key points in a single sentence.

    Output: When outputting this information, use the following structure:
    - [Username]: name of customer who made review
    - [Date]: date of review, written as dd/mm/yyyy
    - [Sentiment]: review sentiment
    - [Key points]: key points of each review in one
    """,
}

content = f"""Summarise these 3 reviews:{reviews} and return your response as a JSON"""

prompt = {
    "role": "user",
    "content": content,
}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

{
  "reviews": [
    {
      "Username": "John123",
      "Date": "15/12/2021",
      "Sentiment": "Positive",
      "Key points": "Delighted with the savings product, fantastic interest rates, helped grow savings significantly, highly recommend"
    },
    {
      "Username": "Sarah77",
      "Date": "28/11/2021",
      "Sentiment": "Negative",
      "Key points": "Disappointed with the savings product, promised returns not as impressive as advertised, fees added up quickly, not worth it"
    },
    {
      "Username": "AlexSmith",
      "Date": "05/01/2022",
      "Sentiment": "Neutral",
      "Key points": "Indifferent about the savings product, similar to any other basic savings account, nothing special but does the job"
    }
  ]
}


## 4. Using Delimiters

Delimiters can help to separate the content and examples from the task description. They can also make it possible to refer to specific parts of your prompt at a later point in the prompt.

A delimiter can be any sequence of characters that usually wouldn’t appear together, for example:

* \>>>>>
* \====
* \####

The number of characters that you use doesn’t matter too much, as long as you make sure that the sequence is relatively unique. Additionally, you can add labels just before or just after the delimiters:

* \START CONTENT>>>>> content <<<<<END CONTENT
* \==== START content END ====
* \#### START EXAMPLES examples #### END EXAMPLES





Ensure delimiters are unique from the rest of your text, otherwise this will confuse the model. An example is given below:

As you can see, we get the output in JSON format. This is quite powerful: when we structure our output as we have done in the previous section, the model can easily convert this into a structured format as shown. Try a different type of format and see what happens.

In [91]:
sentiment_list = f"""
Happy,
Sad,
Disappointed,
Neutral,
Angry
"""

system_message = {
    "role": "system",
    "content": f"""
    You are a Review Summariser. \
    Your job is to process reviews from customers, \
    and summarise them for analysis for the customer review team.

    Group each review into the categories below, delimited by \
    ###:

    ###
    {sentiment_list}
    ###

    """,
}